In [1]:
#!pip install pandas numpy nltk matplotlib seaborn wordcloud
%run 1_data_load.ipynb

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import wordninja

#FinBERT
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

C:\Users\kasim\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kasim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df_news['Time'] = pd.to_datetime(df_news['Time'], format='%d/%m/%Y')
df_news = df_news.rename(columns = {'Time': 'Date'})
print(df_news.dtypes)


Title              object
Tag                object
Date       datetime64[ns]
Content            object
dtype: object


In [4]:
df_price.sort_values('Date', inplace=True)

df_price = df_price.reset_index()
df_price = df_price[['Date', 'Open', 'Close', 'Volume']]
df_price['Date'] = df_price['Date'].dt.tz_localize(None)

df_price.rename(columns={'Close/Last': 'Close'}, inplace=True)
df_price['Date'] = pd.to_datetime(df_price['Date'], format='%m/%d/%Y')

df_price['Prev_change'] = df_price['Close'].pct_change()

df_price['Sma_5'] = df_price['Close'].rolling(window=5).mean() # rolling avg for the last 5 days 

df_price['Volatility'] = df_price['Close'].rolling(window=5).std() # volatility for the last 5 days 

df_price.head(10)

,Date,Open,Close,Volume,Prev_change,Sma_5,Volatility
0,2007-04-02,2425.360107,2422.260010,1784170000,NaN,NaN,NaN
1,2007-04-03,2432.300049,2450.330078,1932030000,0.011588,NaN,NaN
2,2007-04-04,2451.939941,2458.689941,1933300000,0.003412,NaN,NaN
3,2007-04-05,2457.649902,2471.340088,1537800000,0.005145,NaN,NaN
4,2007-04-09,2478.020020,2469.179932,1714880000,-0.000874,2454.360010,19.832749
5,2007-04-10,2468.129883,2477.610107,1834530000,0.003414,2465.430029,10.848758
6,2007-04-11,2477.760010,2459.310059,1957970000,-0.007386,2467.226025,8.125572
7,2007-04-12,2455.719971,2480.320068,1905500000,0.008543,2471.552051,8.202384
8,2007-04-13,2479.689941,2491.939941,1834320000,0.004685,2475.672021,12.246089
9,2007-04-16,2500.550049,2518.330078,1779410000,0.010590,2485.502051,21.761675


In [5]:
# Kursveränderung berechnen
df_price['Change'] = np.round(
            (df_price['Open'] - df_price['Close']) / df_price['Open'], 5
        )

# Label anhand der Kursveränderung
# 1 - Strong gain (>= +20%)
# 2 - Significant gain (+13% to <20%)
# 3 - Moderate gain (+7% to <13%)
# 4 - Stable gain (3% to <+7%)
# 5 - Stable neutral gain (1.5% to <+3%)
# 6 - Neutral (volatility) (-1.5% to <+1.5%)
# 7 - Stable neutral loss (3% to <-1.5%)
# 8 - Stable loss (-7% to <-3%)
# 9 - Moderate loss (-13% to <-7%)
# 10 - Significant loss (-20% to <-13%)
# 11 - Strong loss (< -20%)
conditions = [
    ((df_price['Change'] >= 0.015)), # 1
    ((df_price['Change'] >= 0) & (df_price['Change'] < 0.015)), # 2
    ((df_price['Change'] >= -0.015) & (df_price['Change'] < 0)), # 3
    ((df_price['Change'] < -0.015)) # 4
]
labels = [1, 2, 3, 4]
df_price['Label'] = np.select(conditions, labels)

df_price.head(10)
df_price.groupby(by=['Label']).count()

,Date,Open,Close,Volume,Prev_change,Sma_5,Volatility,Change
Label,,,,,,,,
1,323,323,323,323,323,323,323,323
2,1547,1547,1547,1547,1546,1546,1546,1547
3,1992,1992,1992,1992,1992,1989,1989,1992
4,248,248,248,248,248,248,248,248


In [6]:
# decoupling, case-correction, pyrtial hyphen-resolution
df_news['Content'] = df_news['Content']\
    .str.replace(r'\d+', '', regex=True)\
    .str.replace(r'\.[^.]*$', '', regex=True)

def clean_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', str(text)).lower()
    words = text.split()
    
    tokens = []
    for w in words:
            tokens.extend(wordninja.split(w))
    
    tokens = [w for w in tokens if w not in stop_words and len(w) > 2]
    return tokens

df_news['Cleaned Content'] = df_news['Content'].apply(clean_text)
df_news['Content'] = df_news['Cleaned Content'].apply(lambda x: ' '.join(x))

In [7]:
# Impact Date ist das EInflußdatum einer Nachricht. 
# Wir nehmen an, dass die Nachricht nur am nächsten Arbeitstag einen Einfluß auf die Kursentwicklung hat
df_news['Weekday'] = df_news['Date'].dt.day_name()

df_news['Impact Date'] = np.where(
    df_news['Weekday'] != 'Friday',
    df_news['Date'] + pd.Timedelta(days=1),
    df_news['Date'] + pd.Timedelta(days=3)
)

df_news['Impact Date'] = pd.to_datetime(df_news['Impact Date'], format='%m/%d/%Y')

In [8]:
# Beide DataFrames mit Datumsspalten angleichen
#df_news['date'] = df_news['Time'].dt.date
#df_price['date'] = df_price['Date'].dt.date

# Merge
merged_df = pd.merge(df_news, 
                     df_price[['Date', 'Open', 'Close', 'Label', 'Change', 'Sma_5', 'Volatility', 'Prev_change']], 
                     left_on = 'Impact Date', right_on = 'Date', how='inner'
                    )

#merged_df.groupby(by=['Label']).count()

In [9]:
def simplify_label(label):
    if label < 0:
        return 'negative'
    else:
        return 'positiv'

merged_df['Simple Label'] = merged_df['Change'].apply(simplify_label)
df_news = merged_df

#df_news[df_news['Weekday'].isin(['Friday'])].sort_values(by = "Date", ascending=True)
# max 2023 - 07 - 14
# min 2015 - 06 - 23

In [10]:
merged_df.groupby(by=['Simple Label']).count()
merged_df.to_csv('merged_df.csv', index=False)  

In [11]:
# Only text and labels
tokenized_df = merged_df